In [1]:
from __future__ import annotations

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import AutoTokenizer

from helper import *

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


# Intro

# Load Data

In [3]:
# Reduced data to make it manageable for smaller systems
DATA_FILE: str = '../data/shakespeare_small.txt'

In [4]:
with open(DATA_FILE, 'r') as data_file:
    raw_text = data_file.read()

In [5]:
print(f'Number of characters: {len(raw_text):,}')

Number of characters: 50,085


# Character-Based Generation

## Encode Text into Integer Tokens

### Normalization

> - Skip?
> - lowercase?

In [6]:
def normalize_text(text: str) -> str:
    # Can be multiple actions to normalize text
    normalized_text = text.lower()
    return normalized_text

In [7]:
# TEST: Is your text normalized the way you expected?
# normalized_text = normalize_text(raw_text)
# print(normalized_text)

### Pretokenization

In [8]:
def pretokenize_text(text: str) -> list[str]:
    smaller_pieces = [char for char in text]
    return smaller_pieces

In [9]:
# TEST: Is your (normalized) text pretokenized the way you expected?
# pretokenized_text = pretokenize_text(normalized_text)
# print(pretokenized_text)

### Tokenize

In [10]:
# Combine normalization and pretokenization steps
def tokenize_text(text: str) -> list[str]:
    normalized_text: str = normalize_text(raw_text)
    pretokenized_text: list[str] = pretokenize_text(normalized_text)
    # Characters are already tokens so pretokenized text is already tokenized
    tokenized_text = pretokenized_text
    return tokenized_text

In [11]:
tokenized_text = tokenize_text(raw_text)

### Postprocessing

> Skip

### Encode (Tokens → Integer IDs)

In [12]:
encoded, character_mapping = encode_text(raw_text, tokenize_text)

## Prepare Dataset

In [13]:
n_tokens = character_mapping.n_tokens
dataset_size = len(encoded)

In [14]:
length = 32 # Number of characters
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [15]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [16]:
def generate_text(
    model,
    input_str: str,
    num_chars: int = 100,
    temperature: float = 1.0,
) -> str:
    # Set model into "evaluation mode" (deactivates things like Dropout layers)
    model.eval()
    generated_text = input_str.lower()
    input_tensor = tokens_to_id_tensor(
        tokens=input_str.lower(),
        token_id_mapping=character_mapping.token2id,
    )
    
    with torch.no_grad():
        for _ in range(num_chars):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_char_idx = torch.multinomial(probabilities, 1).item()
            generated_text += character_mapping.id2token(next_char_idx)
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_char_idx]], dtype=torch.long),
                ],
                1,
            )
            
    return generated_text

## Train Model

In [17]:
PHRASE = 'To be or not to be'
epochs = 5

start = start_time()
for epoch in range(epochs):
    # Set model into "training mode"
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    gen_output = generate_text(model, PHRASE, 100)
    print(gen_output)

Epoch 1/5, Loss: 2.2488512074985443
 6.2s (0 0%) 1.9188]
------------------------------------------------------------------------
to be or not to bees' your too.

bfich mean or makich peall arniendmius:
the ase the strour at pear sire
a of are dold
Epoch 2/5, Loss: 1.8962107198306928
 12.5s (1 20%) 1.9033]
------------------------------------------------------------------------
to be or not to beard pelftel's these the: not you, wome.

frathe the rivec.

qheg frosllowly, whad neer you fougf tin
Epoch 3/5, Loss: 1.8158875518713515
 18.8s (2 40%) 1.6527]
------------------------------------------------------------------------
to be or not to be shouds of as witen:
what ne'd
plave cond
to fow hath to out at they not jews to evous the sth and o
Epoch 4/5, Loss: 1.7752618567250407
 25.1s (3 60%) 1.6465]
------------------------------------------------------------------------
to be or not to be sell vi i k catyh all aus that?

first the sely coon btide be inden the cout olden:
fiem partioum 

## Generate Text

In [18]:
output = generate_text(model, 'To be or not to be', 100)
print(output)

to be or not to beith.

detide the clarth:
bork miech grrtne, back.

menenius:
aufice you.

comes, repests first not h


# Token-Based Generation

## Encode Text into Integer Tokens

### Tokenize (Choose)

In [19]:
# Choose a pretrained tokenizer to use
xlmr_model_name = 'xlm-roberta-base'
bert_model_name = 'bert-base-cased'
bert_model_name_uncased = 'bert-base-uncased'

my_tokenizer = AutoTokenizer.from_pretrained(
    bert_model_name_uncased,
)

### Encode (Tokens → Integer IDs)

In [20]:
encoded, token_mapping = encode_text_from_tokenizer(
    text=raw_text,
    tokenizer=my_tokenizer,
)

## Prepare Dataset

In [21]:
n_tokens = token_mapping.n_tokens
dataset_size = len(encoded)

In [22]:
length = 16  # Tokens 
batch_size = 32

train_dataset = ShakespeareDataset(encoded, length)
train_loader = DataLoader(
    train_dataset,
    shuffle=True,
    batch_size=batch_size,
)

## Define Model

In [23]:
model = build_model(n_tokens)
model.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters())

## Define Text Genaration

In [24]:
def generate_text(
    tokenizer,
    model,
    input_str: str,
    num_tokens: int = 100,
    temperature: float = 1.0,
) -> str:
    # Set model into "evaluation mode" (deactivates things like Dropout layers)
    model.eval()
    tokenized_text = tokenize_text_from_tokenizer(
        tokenizer=tokenizer,
        text=input_str,
    )
    input_tensor = tokens_to_id_tensor(
        tokens=tokenized_text,
        token_id_mapping=token_mapping.token2id,
    )

    generated_text = []
    with torch.no_grad():
        for _ in range(num_tokens):
            output = model(input_tensor.to(device))
            probabilities = nn.functional.softmax(
                output[0, -1] / temperature,
                dim=0,
            )
            next_token_idx = torch.multinomial(probabilities, 1).item()
            generated_text.append(token_mapping.id2token(next_token_idx))
            input_tensor = torch.cat(
                [
                    input_tensor,
                    torch.tensor([[next_token_idx]], dtype=torch.long),
                ],
                1,
            )
    # Convert to text again
    output_ids = tokenizer.convert_tokens_to_ids(generated_text)
    output_str = input_str + ' ' + tokenizer.decode(output_ids)
    return output_str

## Train Model

In [25]:
epochs = 5

start = start_time()
for epoch in range(epochs):
    # Set model into "training mode"
    model.train()
    total_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        output = model(X_batch.to(device))
        loss = criterion(output.transpose(1, 2), y_batch.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    
    print(f'Epoch {epoch + 1}/{epochs}, Loss: {total_loss / len(train_loader)}')
    print('[%s (%d %d%%) %.4f]' % (time_since(start), epoch, epoch / epochs * 100, loss))
    print('-'*72)
    output = generate_text(my_tokenizer, model, 'To be or not to be', 30)
    print(output)


Epoch 1/5, Loss: 6.218242396983497
 2.7s (0 0%) 5.9829]
------------------------------------------------------------------------
To be or not to be spectacle to.h '.,,, your : not ci we be shouts whole my thyer kisses.us and - seem or they : of
Epoch 2/5, Loss: 5.566582839854442
 5.4s (1 20%) 5.3855]
------------------------------------------------------------------------
To be or not to be ##um trncy mali man the alias, ;'s shouldus :h your receive and with, let what help the what of carries ll the painting
Epoch 3/5, Loss: 5.034900190766421
 8.1s (2 40%) 4.6247]
------------------------------------------------------------------------
To be or not to be tired thoughts to take him withus lead met relievedidius :ces? not our then your fai din volum : off hes for myrti
Epoch 4/5, Loss: 4.6204903119962
 10.9s (3 60%) 4.2314]
------------------------------------------------------------------------
To be or not to be speak : he serve it entire to depends, the business thousandub and the con

## Generate Text

In [26]:
output = generate_text(my_tokenizer, model, 'To be or not to be', 30)
print(output)

To be or not to be orangeec well in off true many would wish virgiliaia : come not so! second citizen : come, that'd the parties to good?


# Comparison Between Generation